In [ ]:
import geopandas as gpd
import polars as pl
import polars_st as st
import pyogrio

In [ ]:
def to_st_gdf(gdp_gdf: gpd.GeoDataFrame) -> st.GeoDataFrame:
    st_gdf = st.from_geopandas(gdp_gdf.to_crs(4326)).with_columns(st.geom("geometry").st.set_srid(4326))
    return st_gdf


def to_gpd_gdf(st_gdf: st.GeoDataFrame) -> gpd.GeoDataFrame:
    gpd_gdf = st_gdf.st.to_geopandas().set_crs(4326)
    return gpd_gdf

In [ ]:
sa1_file = "temp/sa1_processed.shp"
sa1 = (
    pyogrio.read_dataframe(sa1_file)
    .pipe(to_st_gdf)
    .select(pl.col("SA1_CODE21").cast(pl.UInt64).alias("sa1"), "geometry")
)

In [ ]:
sa2 = (
    sa1.with_columns(
        pl.col("sa1").cast(pl.String).str.slice(0, 9).alias("sa2"),
        pl.col("sa1").cast(pl.String).str.slice(0, 5).alias("sa3"),
        pl.col("sa1").cast(pl.String).str.slice(0, 3).alias("sa4"),
        pl.col("sa1").cast(pl.String).str.slice(0, 1).alias("s/t"),
    )
    .group_by("sa3")
    .agg(st.geom("geometry").st.union_all())
)
sa2

In [ ]:
sa2.pipe(to_gpd_gdf).explore()